In [1]:
%matplotlib inline

# General packages for system, time, etc
import os, time, csv
import datetime
from datetime import date
import glob
import feather  # for compiling files

# scitnific computing and plotting
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

# HDDM related packages
import pymc as pm
import hddm
import kabuki
import arviz as az
print("The current HDDM version is: ", hddm.__version__)
print("The current PyMC version is: ", pm.__version__)
print("The current ArviZ version is: ", az.__version__)

# parallel processing related
from p_tqdm import p_map
from functools import partial

/opt/conda/lib/python3.8/site-packages/IPython/parallel.py:12: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  warn("The `IPython.parallel` package has been deprecated since IPython 4.0. "


The current HDDM version is:  0.8.0
The current PyMC version is:  2.3.8
The current ArviZ version is:  0.11.4


In [2]:
from HDDMarviz import HDDMarviz

In [3]:
data_cavanagh = hddm.load_csv(os.path.join(os.path.dirname(hddm.__file__), 
                                           'examples', 
                                           'cavanagh_theta_nn.csv'))
data_cavanagh.tail()

,subj_idx,stim,rt,response,theta,dbs,conf
3983,13,LL,1.450,0.0,-1.237166,0,HC
3984,13,WL,0.711,1.0,-0.377450,0,LC
3985,13,WL,0.784,1.0,-0.694194,0,LC
3986,13,LL,2.350,0.0,-0.546536,0,HC
3987,13,WW,1.250,1.0,0.752388,0,HC


In [4]:
%%time

# define a function to run model in parallel

# M0_0: base model: simplified
def ms0(id, df=None, samples=None, burn=None, thin=1, save_name="ms0"): 
    # for HDDM models, adding this print func can prevent warnings that would occur
    #  when running parallel processing in jupyter
    print('running chain {:d} for model {}'.format(id, save_name))
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id    
    m = hddm.HDDM(df)
    m.find_starting_values()
    m.sample(samples, burn=burn, thin=thin, dbname=dbname, db='pickle') # it's neccessary to save the model data
    m.save(mname)
    
    return m

# M1: base model: full model
def ms1(id, df=None, samples=None, burn=None, thin = 1, save_name="ms1"): 
    print('running chain {:d} for model {}'.format(id, save_name))
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id    
    m = hddm.HDDM(df, include=['z', 'sv', 'sz', 'st'])
    m.find_starting_values()
    m.sample(samples, burn=burn, thin=thin, dbname=dbname, db='pickle') # it's neccessary to save the model data
    m.save(mname)
    
    return m


# M2: treat within-subj as between-subj: full model
def ms2(id, df=None, samples=None, burn=None, thin = 1, save_name="ms2"): 
    print('running chain {:d} for model {}'.format(id, save_name))
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id    
    m = hddm.HDDM(df, include=['z', 'sv', 'st', 'sz'], 
                  depends_on={'v': 'conf'})
    m.find_starting_values()
    m.sample(samples, burn=burn, dbname=dbname, db='pickle') # it's neccessary to save the model data
    m.save(mname)
    
    return m


# M3: regression model (varying intercept)
def ms3(id, df=None, samples=None, burn=None, thin = 1, save_name="ms3"): 
    print('running chain {:d} for model {}'.format(id, save_name))
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id   
    m = hddm.HDDMRegressor(df,  
                           "v ~ C(conf, Treatment('LC'))", 
                           group_only_regressors=True,
                           keep_regressor_trace=True,
                           include=['z', 'sv', 'st', 'sz'])
    m.find_starting_values()
    m.sample(samples, burn=burn, thin=thin, dbname=dbname, db='pickle') # it's neccessary to save the model data
    m.save(mname)
    
    return m

# M4: regression model (varying intercept and slope)
def ms4(id, df=None, samples=None, burn=None, thin = 1, save_name="ms4"): 
    print('running chain {:d} for model {}'.format(id, save_name))
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id   
    m = hddm.HDDMRegressor(df,
                           "v ~ C(conf, Treatment('LC'))", 
                           group_only_regressors=False,
                           keep_regressor_trace=True,
                           include=['z', 'sv', 'st', 'sz'])
    m.find_starting_values()
    m.sample(samples, burn=burn, thin=thin, dbname=dbname, db='pickle') # it's neccessary to save the model data
    m.save(mname)
    
    return m

# M5: regression model + theta as an additional predictor of `a`
def ms5(id, df=None, samples=None, burn=None, thin = 1, save_name="ms5"): 
    print('running chain {:d} for model {}'.format(id, save_name))
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id
    m = hddm.HDDMRegressor(df,
                           "a ~ theta:C(conf, Treatment('LC'))",
                           depends_on={'v': 'conf'},
                           group_only_regressors=False,
                           keep_regressor_trace=True,
                           include=['z', 'sv', 'st', 'sz'])
    m.find_starting_values()
    m.sample(samples, burn=burn, thin=thin, dbname=dbname, db='pickle') # it's neccessary to save the model data
    m.save(mname)
    
    return m

# M6: Regression for both parameters
def ms6(id, df=None, samples=None, burn=None, thin = 1, save_name="ms6"): 
    print('running chain {:d} for model {}'.format(id, save_name))
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id
    a_reg = {'model': "a ~ theta:C(conf, Treatment('LC'))", 'link_func': lambda x: x}
    v_reg = {'model': "v ~ C(conf, Treatment('LC'))", 'link_func': lambda x: x}
    reg_descr = [a_reg, v_reg]
    
    m = hddm.HDDMRegressor(df,
                           reg_descr,
                           group_only_regressors=False,
                           keep_regressor_trace=True,
                           include=['z', 'sv', 'st', 'sz'])
    m.find_starting_values()
    m.sample(samples, burn=burn, thin=thin, dbname=dbname, db='pickle') # it's neccessary to save the model data
    m.save(mname)
    
    return m

# M7: Regression for both parameters
def ms7(id, df=None, samples=None, burn=None, thin = 1, save_name="ms7"): 
    print('running chain {:d} for model {}'.format(id, save_name))
    import hddm
    
    dbname = save_name + '_chain_%i.db'%id 
    mname  = save_name + '_chain_%i'%id
    a_reg = {'model': "a ~ theta:C(conf, Treatment('LC')):C(dbs, Treatment('0'))", 'link_func': lambda x: x}
    v_reg = {'model': "v ~ C(conf, Treatment('LC'))", 'link_func': lambda x: x}
    reg_descr = [a_reg, v_reg]
    
    m = hddm.HDDMRegressor(df,
                           reg_descr,
                           group_only_regressors=False,
                           keep_regressor_trace=True,
                           include=['z', 'sv', 'st', 'sz'])
    m.find_starting_values()
    m.sample(samples, burn=burn, thin=thin, dbname=dbname, db='pickle') # it's neccessary to save the model data
    m.save(mname)
    
    return m

CPU times: user 2 µs, sys: 3 µs, total: 5 µs
Wall time: 8.34 µs


In [5]:
samples = 300  # Cavanagh et al. 2011 used 30,000 and 10, 000 burn.
burn = 100     
nppc = 50     # 1000 samples for posterior predictive, super slow
thin = 1
chains = 4
test_mode = False
savefile=True
savetag = "tmp"

In [6]:
%%time

## total time: 5h 19s when test_mode is False

## Step 1: run models in parallel
file_path = "/home/jovyan/hddm/temp/"

if test_mode:
    model_func = ms0

else: 
    model_func = [ms0,  
                  ms4]

CPU times: user 1e+03 ns, sys: 2 µs, total: 3 µs
Wall time: 6.91 µs


In [7]:
callable(model_func)

False

note: here we have problems when running parallel processing in jupyter, the issue here is similar to here: https://github.com/tqdm/tqdm/issues/485. I need to figure it out which function can be hacked with `print()`.

Note: when model is ms0, there will be a warning about msg_id

In [8]:
# model = model_func[0]
# m_key = model.__name__  # get the model function's name
# save_name_m = m_key + "_" + savetag
# print(save_name_m)

# ms_tmp = p_map(partial(model,
#                        df=data_cavanagh, 
#                        samples=samples,
#                        burn=burn,
#                        thin=thin, 
#                        save_name=save_name_m),
#                    range(chains))

# xdata_post_pred = [] # define an empty dict    

# xdata_post_pred = p_map(partial(post_pred_gen, samples = nppc), models['ms0'])

In [ ]:
models, InfData = HDDMarviz(data=data_cavanagh, 
                            model_func=model_func,
                            samples=samples, 
                            nppc=nppc, 
                            burn=burn, 
                            thin=thin, 
                            chains=chains, 
                            savefile=savefile,
                            savetag=savetag)

start model fitting for ms0
running chain 0 for model ms0_tmp

running chain 2 for model ms0_tmprunning chain 1 for model ms0_tmp

  0%|          | 0/4 [00:00<?, ?it/s]


running chain 3 for model ms0_tmp


/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 -                 4%                  ] 12 of 300 complete in 0.5 secc [-                 4%                  ] 13 of 300 complete in 0.5 sec[-                 4%                  ] 12 of 300 complete in 0.5 sec[--                7%                  ] 23 of 300 complete in 1.0 sec[---               8%                  ] 24 of 300 complete in 1.0 sec[--                7%                  ] 23 of 300 complete in 1.0 sec[---               8%                  ] 24 of 300 complete in 1.1 sec[----             11%                  ] 35 of 300 complete in 1.5 sec[----             11%                  ] 34 of 300 complete in 1.5 sec[----             12%                  ] 36 of 300 complete in 1.6 sec[----             11%                  ] 35 of 300 complete in 1.6 sec[-----            15%                  ] 45 of 300 complete in 2.0 sec[-----            15%                  ] 47 of 300 complete in 2.1 sec[-----            15%                  ] 47 of 300 complete in 2.1 sec[-----            

...

/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.8/site-packages/scipy/optimize/optimize.py:2215: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 --                7%                  ] 22 of 300 complete in 22.2 sec                  0%                  ] 2 of 300 complete in 0.9 sec[                  0%                  ] 2 of 300 complete in 1.0 sec[                  0%                  ] 2 of 300 complete in 0.9 sec[                  1%                  ] 3 of 300 complete in 1.8 sec[                  1%                  ] 3 of 300 complete in 1.9 sec[                  1%                  ] 3 of 300 complete in 2.0 sec[                  1%                  ] 3 of 300 complete in 1.9 sec[                  1%                  ] 4 of 300 complete in 2.9 sec[                  1%                  ] 4 of 300 complete in 2.9 sec[                  1%                  ] 4 of 300 complete in 2.9 sec[                  1%                  ] 4 of 300 complete in 3.1 sec[                  1%                  ] 5 of 300 complete in 3.8 sec[                  1%                  ] 5 of 300 complete in 4.0 sec[                  1%            

 [---              10%                  ] 31 of 300 complete in 31.4 sec[----             10%                  ] 32 of 300 complete in 31.5 sec[----             10%                  ] 32 of 300 complete in 31.5 sec[---              10%                  ] 31 of 300 complete in 31.5 sec[----             11%                  ] 33 of 300 complete in 32.4 sec[----             11%                  ] 33 of 300 complete in 32.5 sec[----             10%                  ] 32 of 300 complete in 32.5 sec[----             10%                  ] 32 of 300 complete in 32.5 sec[----             11%                  ] 34 of 300 complete in 33.6 sec[----             11%                  ] 34 of 300 complete in 33.6 sec[----             11%                  ] 33 of 300 complete in 33.6 sec[----             11%                  ] 33 of 300 complete in 33.6 sec[----             11%                  ] 35 of 300 complete in 34.6 sec[----             11%                  ] 35 of 300 complete in 34.7 sec[----

 [-------          19%                  ] 59 of 300 complete in 64.2 sec[-------          19%                  ] 59 of 300 complete in 64.5 sec[-------          20%                  ] 62 of 300 complete in 64.6 sec[-------          20%                  ] 62 of 300 complete in 65.1 sec[-------          20%                  ] 60 of 300 complete in 65.6 sec[-------          20%                  ] 60 of 300 complete in 65.7 sec[-------          21%                  ] 63 of 300 complete in 65.7 sec[-------          21%                  ] 63 of 300 complete in 66.2 sec[-------          20%                  ] 61 of 300 complete in 66.8 sec[--------         21%                  ] 64 of 300 complete in 66.7 sec[-------          20%                  ] 61 of 300 complete in 67.0 sec[--------         21%                  ] 64 of 300 complete in 67.3 sec[-------          20%                  ] 62 of 300 complete in 67.8 sec[--------         21%                  ] 65 of 300 complete in 67.9 sec[----

 [-----------      30%                  ] 92 of 300 complete in 96.7 sec[-----------      30%                  ] 90 of 300 complete in 97.0 sec[-----------      29%                  ] 87 of 300 complete in 97.1 sec[-----------      29%                  ] 89 of 300 complete in 97.3 sec[-----------      30%                  ] 91 of 300 complete in 97.9 sec[-----------      31%                  ] 93 of 300 complete in 97.9 sec[-----------      29%                  ] 88 of 300 complete in 98.3 sec[-----------      30%                  ] 90 of 300 complete in 98.4 sec[-----------      31%                  ] 94 of 300 complete in 99.0 sec[-----------      30%                  ] 92 of 300 complete in 99.2 sec  [-----------      30%                  ] 91 of 300 complete in 99.5 sec[-----------      29%                  ] 89 of 300 complete in 99.5 sec[-----------      31%                  ] 93 of 300 complete in 100.2 sec[------------     31%                  ] 95 of 300 complete in 100.0 sec[

In [ ]:
models

## Bayesian modelling with `ArviZ`

### Model diagnosis

#### Visual inspection

In [ ]:
InfData

In [ ]:
# from arviz.utils import Numba
# Numba.disable_numba()
# Numba.numba_flag
az.plot_trace(InfData['ms0'])

In [ ]:
#     using regex to select var_names that start with "a" 
# and do not contain either "subj" or "std"

az.plot_trace(InfData['ms4'], var_names=("^a(?!.*(subj|std))"), filter_vars='regex')

#### Using `az.summary()` to check $\hat{R}$ and ESS

In [ ]:
ms0_summary = az.summary(InfData['ms0'])
ms0_summary.sort_values('r_hat')

In [ ]:
ms4_summary = az.summary(InfData['ms4'])
ms4_summary.sort_values('r_hat')

### Model comparison and selection

#### DIC

In [ ]:
%%time

tmp_dic = []
indx_name = []
for m_key, model in models.items():
#     print(len(models[key]))
    m_tmp = kabuki.utils.concat_models(model)
    tmp_dic.append(m_tmp.dic)
    indx_name.append(m_key)
#     print(m_key + "'s DIC: ", m_tmp.dic) # model 4 has the lowest DIC
    
comp_dic = pd.DataFrame(tmp_dic, index=indx_name, columns=['dic'])
comp_dic = comp_dic.sort_values(by=['dic'])
comp_dic = comp_dic.reset_index()
comp_dic.rename(columns={'index':'rank'}, inplace=True)
comp_dic

#### PSIS-LOO-CV

In [ ]:
comp_loo = az.compare(InfData, ic='loo')
comp_loo

####  WAIC

In [ ]:
comp_waic = az.compare(InfData, ic='waic')
comp_waic

#### Posterior predictive check

In [ ]:
az.plot_ppc(InfData['ms0'])

In [ ]:
az.plot_ppc(InfData['ms4'])

In [ ]:
from plot_ppc_by_cond import plot_ppc_by_cond

In [ ]:
plot_ppc_by_cond(data = InfData['ms4'],
                 or_d = data_cavanagh,
                 subjs = [3],
#                  conds = ['conf'],
                 conds = ['conf','dbs'],
                 colors = ['r', 'k', 'b'],
                 num_pp_samples=100,
                 random_seed = 7,
                 alpha = 0.2,
                 grid = [2,2],
                 var_names=['rt'])

In [ ]:
plot_ppc_by_cond(data = InfData['ms0'],
                 or_d = data_cavanagh,
                 subjs = [3],
#                  conds = ['conf'],
                 conds = ['conf','dbs'],
                 colors = ['r', 'k', 'b'],
                 num_pp_samples=100,
                 random_seed = 7,
                 alpha = 0.2,
                 grid = [2,2],
                 var_names=['rt'])

## Statistical Inference

In [ ]:
# using regex to select var_names that start with "a_theta" and do not contain either "subj" or "std"
az.plot_posterior(InfData['ms4'], 
                  var_names=("^v_(?!.*(subj|std))"), 
                  filter_vars='regex',
                  grid = [2, 2], 
                  kind = 'hist',
                  hdi_prob = 0.95,
                  rope = [0.45, 0.55], 
                  rope_color = 'r')